# Reporte de producción mensual

In [ ]:
import mysql.connector
import pandas as pd
from datetime import datetime, timedelta

In [2]:
def format_timedelta_hh_mm_ss(td: timedelta) -> str:
    total_seconds = int(td.total_seconds())
    hours, remainder = divmod(total_seconds, 3600)
    minutes, seconds = divmod(remainder, 60)
    return f"{hours:02d}:{minutes:02d}:{seconds:02d}"

config = {
  'user': 'operario',
  'password': '',
  'host': '192.168.0.12', # O la IP de tu servidor
  'database': 'dbp8100'
}

def conectar_mysql():
    return mysql.connector.connect(**config)
    
def obtener_df(fecha_inicio, fecha_fin):
    db = conectar_mysql()
    cur = db.cursor()
    cur.execute('''
                SELECT
                    subconsulta.Codigo,
                    subconsulta.Nombre,
                    SUM(subconsulta.Dosificado) AS Total
                FROM
                    (   SELECT
                            tareaseje.NroID,
                            tareaseje.Fecha AS Fecha,
                            tareaseje.Hora AS Hora,
                            formulas.NroID AS IDF,
                            formulas.Codigo AS Codigo,
                            formulas.Nombre AS Nombre,
                            tareaseje.Set AS Programado,
                            (SELECT SUM(dcaptura.Valor) FROM dbp8100.dcaptura WHERE dcaptura.IDT = tareaseje.NroID) AS Dosificado,
                            tareaseje.Tiempo AS Tiempo
                        FROM
                            dbp8100.tareaseje AS tareaseje
                        JOIN
                            dbp8100.formulas AS formulas ON formulas.NroID = tareaseje.IDF
                        WHERE
                            (tareaseje.Fecha = %s AND tareaseje.Hora >= '22:00:00') OR
                            (tareaseje.Fecha > %s AND tareaseje.Fecha <= %s AND tareaseje.Hora < '22:00:00')
                    ) AS subconsulta
                -- Agrupamos por el nombre de la fórmula para sumar los valores dosificados
                GROUP BY
                    subconsulta.Nombre;
                ''',(fecha_inicio, fecha_inicio, fecha_fin))

    resultados = cur.fetchall()
    # Obtener los nombres de las columnas
    columnas = [desc[0] for desc in cur.description]

    # Crear un DataFrame de pandas
    df = pd.DataFrame(resultados, columns=columnas)

    return df

In [4]:
df = obtener_df('2025-08-01', '2025-08-31')
df

,Codigo,Nombre,Total
0,FP0027,(LA MALVINA) DESARROLLO TERNEROS MAIZ SIN SAL,15064.00
1,60-020,AP-40....,3999.90
2,FP0012,CABAÑA PRESENTACIÓN (COMAGRO),8031.40
3,51-004,CERDAS GESTACIÓN (MAXIMIX),1199.00
4,FC820,CERDAS GESTACIÓN (LA MALVINA) MAXIMIX.,6014.30
5,52-005,CERDAS LACTANCIA (MAXIMIX).,1201.50
6,FC913,CERDAS LACTANCIA (MAXIMIX) LA MALVINA..,3996.40
7,09-141,CRE. MELAZA 10% A.A. (Los Roanos),58432.45
8,09-143,CREC. MELAZA 12 % A.A....,467983.55
9,CP-001,CRECIMIENTO POWER,7998.85


In [5]:
df.to_excel('08.xlsx', index=False)